## HTTP Router using a Trie
For this exercise we are going to implement an HTTPRouter like you would find in a typical web server using the Trie data structure we learned previously.

There are many different implementations of HTTP Routers such as regular expressions or simple string matching, but the Trie is an excellent and very efficient data structure for this purpose.

The purpose of an HTTP Router is to take a URL path like "/", "/about", or "/blog/2019-01-15/my-awesome-blog-post" and figure out what content to return. In a dynamic web server, the content will often come from a block of code called a handler.

![title](https://s3.amazonaws.com/video.udacity-data.com/topher/2017/August/598b5e9b_2-5-managing-app-location-with-react-router-2x/2-5-managing-app-location-with-react-router-2x.jpg)

First we need to implement a slightly different Trie than the one we used for autocomplete. Instead of simple words the Trie will contain a part of the http path at each node, building from the root node /

In addition to a path though, we need to know which function will handle the http request. In a real router we would probably pass an instance of a class like Python's [SimpleHTTPRequestHandler](https://docs.python.org/3/library/http.server.html#http.server.SimpleHTTPRequestHandler) which would be responsible for handling requests to that path. For the sake of simplicity we will just use a string that we can print out to ensure we got the right handler

We could split the path into letters similar to how we did the autocomplete Trie, but this would result in a Trie with a very large number of nodes and lengthy traversals if we have a lot of pages on our site. A more sensible way to split things would be on the parts of the path that are separated by slashes ("/"). A Trie with a single path entry of: "/about/me" would look like:

(root, None) -> ("about", None) -> ("me", "About Me handler")

We can also simplify our RouteTrie a bit by excluding the suffixes method and the endOfWord property on RouteTrieNodes. We really just need to insert and find nodes, and if a RouteTrieNode is not a leaf node, it won't have a handler which is fine.

Next we need to implement the actual Router. The router will initialize itself with a RouteTrie for holding routes and associated handlers. It should also support adding a handler by path and looking up a handler by path. All of these operations will be delegated to the RouteTrie.

Hint: the RouteTrie stores handlers under path parts, so remember to split your path around the '/' character

Bonus Points: Add a not found handler to your Router which is returned whenever a path is not found in the Trie.

More Bonus Points: Handle trailing slashes! A request for '/about' or '/about/' are probably looking for the same page. Requests for '' or '/' are probably looking for the root handler. Handle these edge cases in your Router.

In [338]:
# A RouteTrieNode will be similar to our autocomplete TrieNode... 
# with one additional element, a handler.
class RouteTrieNode:
    def __init__(self, handler=None):
        # Initialize the node with children as before, plus a handler
        self.handler = handler
        self.children = {}
    def __repr__(self):
        return f"{self.__class__.__name__}(handler={self.handler}, children= {self.children})"

In [312]:
class RouteTrie:
    def __init__(self, root_handler=None):
        self.root = RouteTrieNode(handler=root_handler)
        
    def insert(self, path_parts, handler):
        current_node = self.root
        for part in path_parts:
            if part not in current_node.children:
                current_node.children[part] = RouteTrieNode()
            current_node = current_node.children[part]
            
        current_node.handler = handler

    def find(self, path_parts):
        current_node = self.root
        for part in path_parts:
            if part not in current_node.children:
                return None
            else:
                current_node = current_node.children[part]
        handler = current_node.handler
        return handler
        

In [319]:
class Router:
    def __init__(self, root_handler, not_found_handler):
        self.rt = RouteTrie(root_handler=root_handler)
        self.not_found_handler = not_found_handler
        # Create a new RouteTrie for holding our routes
        # You could also add a handler for 404 page not found responses as well!

    def add_handler(self, path, handler):
        path_parts = self.split_path(path)
        self.rt.insert(path_parts, handler)

    def lookup(self, path):
        if path == "/":
            return root_handler
        path_parts = self.split_path(path)
        handler = self.rt.find(path_parts)
        if handler is None:
            return self.not_found_handler
        else:
            return handler
    def split_path(self, path):
        return path.strip("/").split("/")


### Test Cases


In [333]:

# create the router and add a route
router = Router("root handler", "404. Thats an error.")
router.add_handler("/home/about", "about handler")  # add a route
# some lookups with the expected output
print(router.lookup("/")) # should print 'root handler'
print(router.lookup("/home")) # should print 'not found handler' or None if you did not implement one
print(router.lookup("/home/about")) # should print 'about handler'
print(router.lookup("/home/about/")) # should print 'about handler' or None if you did not handle trailing slashes
print(router.lookup("/home/about/me")) # should print 'not found handler' or None if you did not implement one

you've found home
404. Thats an error.
about handler
about handler
404. Thats an error.


## Write up
`.lookup()` executes in `O(n)` time in the worst case, where n is the number of path-parts in the path we're looking up. This is because we still need to look through every single part of the path until we get to ensure that it exists. I figure that the best case (from a time efficiency standpoint) is when the first part of the path doesn't exist in the children of the root of the trie and we return `None` (or our '404 error not found' message) after just one attempt to traverse down th trie.

`.add_handler()` has a time efficiency of of `O(n)` in the worst case scenario. We first need to iterate through the path to split it into chunks at the slash marks, and then we also need to use `insert()`, which operates in `O(n)` time for a trie.

Similar to the answer for problem 5, I believe that in the worst-case scenario, the space efficiency is `O(n)`, because no path would be a prefix to any other path and thus each part of each path would have a separate node. This is unlikely in practice, however. The whole point of using a trie is to take advantage of a situation where many keys share the same prefix. 

In [ ]:
a